## Import Libraries

In [1]:
import json
import pandas as pd

## Load Files for Parsing

- Transform json file to dataframe
- Transform info file to dataframe
- Merge json dataframe with info dataframe

In [2]:
json_data_path = "/Users/claudia/Downloads/data.json"
info_data_path = "/Users/claudia/Downloads/data.info"
data = [json.loads(line) for line in open(json_data_path, 'r')]
with open(info_data_path, 'r') as f:
    info = f.read().splitlines()

In [5]:
## function to get key of a dictionary
def get_key(dictionary):
    key_object = dictionary.keys()
    key = list(key_object)[0]
    return key

## function to help concatenate columns to get transcript, position, nucleotides
def concat_col(transcript, position, nucleotide, n):
    t_df = pd.DataFrame([transcript]*n)
    p_df = pd.DataFrame([position]*n)
    nu_df = pd.DataFrame([nucleotide]*n)
    n_df = pd.DataFrame([n]*n)

    ## concat columns together
    final_df = pd.concat([t_df, p_df, nu_df, n_df], axis = 1)
    final_df.columns = ['transcript', 'position', 'nucleotides', 'reads_count']
    return final_df

## function to parse line in json file
def parse_line(line):
    ## get transcript
    t = get_key(line)

    ## get position
    p = get_key(line[t])

    ## get nucleotide seq
    nu = get_key(line[t][p])

    ## get number of reads
    reads_count = len(line[t][p][nu])

    ## get dataframe of list of reads
    reads = pd.DataFrame(line[t][p][nu])

    ## concat columns together to get transcript, position, nucleotides and all dwelling time, std, mean
    df = pd.concat([concat_col(t, p, nu, reads_count), reads], axis = 1)
    df.columns = ['transcript', 'position', 'nucleotides', 'reads_count', 'dwellingtime_-1', 'std_-1', 'mean_-1', 'dwellingtime_0', 'std_0', 'mean_0', 'dwellingtime_+1', 'std_+1', 'mean_+1']

    return df

In [6]:
parse_line(data[0])

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,ENST00000000233,244,AAGACCA,185,0.02420,4.45,124.0,0.00564,4.45,128.0,0.00365,2.41,73.1
181,ENST00000000233,244,AAGACCA,185,0.00557,5.21,126.0,0.00266,1.85,129.0,0.00896,8.68,79.0
182,ENST00000000233,244,AAGACCA,185,0.00764,3.90,124.0,0.00630,5.38,123.0,0.01200,2.69,82.6
183,ENST00000000233,244,AAGACCA,185,0.00863,3.46,121.0,0.00894,6.27,126.0,0.00598,1.52,78.6


In [7]:
## parse all lines in json file into dataframe for concatenation
reads_df = [parse_line(data[i]) for i in range(len(data))]
data_df = pd.concat(reads_df, axis = 0)

In [8]:
print(f"Number of rows in data_df = {len(data_df)}")
data_df.head()

Number of rows in data_df = 11027106


,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2


In [9]:
## transform info data into dataframe
info_list = [info[i].split(",") for i in range(len(info))]
info_df = pd.DataFrame(info_list[1:]) 
info_df.columns = info_list[0]
info_df.head()

,gene_id,transcript_id,transcript_position,label
0,ENSG00000004059,ENST00000000233,244,0
1,ENSG00000004059,ENST00000000233,261,0
2,ENSG00000004059,ENST00000000233,316,0
3,ENSG00000004059,ENST00000000233,332,0
4,ENSG00000004059,ENST00000000233,368,0


In [10]:
## left join data_df and info_df to get features and labels
complete_df = data_df.merge(info_df, how = "left", left_on = ["transcript", "position"], right_on = ["transcript_id", "transcript_position"])
complete_df.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,gene_id,transcript_id,transcript_position,label
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,ENST00000000233,244,0
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,ENST00000000233,244,0
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,ENST00000000233,244,0
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,ENST00000000233,244,0
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,ENST00000000233,244,0


In [12]:
## drop duplicate transcript_id and transcript_position column
complete_df.drop(['transcript_id', 'transcript_position'],  axis = 1, inplace = True)
complete_df.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,gene_id,label
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


### Saving complete_df into parquet files

In [22]:
## function to breakdown dataframe into smaller sizes and save it
def save_file(df, filename, nrows = 5000000):
    total_rows = len(df)
    start, stop, count = 0, nrows, 1

    ## use while loop to break the dataframe into smaller dataframes
    while stop < total_rows:
        print(start, stop)
        temp_df = pd.DataFrame(df.iloc[start:stop, :])
        fname = f"{filename}_{count}.parquet"
        temp_df.to_parquet(fname)
        print(f"Saved a file called {fname}")
        count += 1
        start += nrows
        stop += nrows
    
    stop = total_rows
    print(start, stop)
    temp_df = pd.DataFrame(df.iloc[start:stop, :])
    fname = f"{filename}_{count}.parquet"
    temp_df.to_parquet(fname)
    print(f"Saved a file called {fname}")

In [23]:
save_file(complete_df, "merged_data_v2")

0 5000000
Saved a file called merged_data_v2_1.parquet
5000000 10000000
Saved a file called merged_data_v2_2.parquet
10000000 11027106
Saved a file called merged_data_v2_3.parquet
